# Instalando bibliotecas

In [1]:
""" Installing libraries """
%pip install pandas==2.3.2 matplotlib==3.10.6 seaborn==0.13.2 scikit-learn==1.7.1 numpy==2.2.6 pyarrow==21.0.0 torch==2.8.0

Note: you may need to restart the kernel to use updated packages.


# Importando bibliotecas (externas e próprias)

In [2]:
""" Importing libraries """

import sys
import os

# Add the parent directory to sys.path so 'Modules' can be imported
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# Our modules
from Modules.loading.read_parquet import read_parquet_file 
from Modules.preprocessing.onehot import one_hot_encode_parquet
from Modules.models.make_dataset import SingleSeriesDataset, MultiSeriesDataset
from Modules.models.NBeats import NBeatsBlock, NBeats
from Modules.models.WMAPELoss import WMAPELoss
from Modules.models.training import train_model
from Modules.models.test import soft_test
from Modules.models.test import hard_test
from Modules.models.test import hard_test1

ModuleNotFoundError: No module named 'Modules.models.make_dataset'

# Definição dos hiper-parâmetros

In [ ]:
""" Defining hyper-parameters """
# Neural Network Global Parameters
input_size = 7  # Number of past days to use as input
output_size = 1  # Number of future days to predict
batch_size = 14  # Batch size for training

n_layers = 4  # Number of layers in the N-BEATS model
hidden_size = 32  # Number of hidden units in each layer

# Training parameters
learning_rate = 1e-3 # Learning rate for the optimizer
epochs = 1000  # Number of training epochs (iterations over the entire dataset)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

# Importação do Dataset

É interessante dividir o treino em batches (mini-conjuntos de treino). Cada batch possui o tamanho de input size, seguindo a ordem cronológica de vendas dentro daquela janela de dias. No entanto, durante o treinamento é **ESSENCIAL** que a escolha do próximo batch seja aleatória.

Ex.: Inicia o treino por 21-27 jul e prevê 28, depois pula para 02-08 fev para prever 03. Esse processo deve ser repetido até todos os dados serem treinados, finalizando **01 epoch**.

O número de **epochs** diz o número total de iterações do modelo com relação ao dataset inteiro.

Sobre a composição da janela de input dentro de um batch, existem duas abordagens:

1) Treinar em cada janela todas as séries (pense que cada par produto-loja x tempo representa uma série temporal dentro daquele período). Esse modelo é bem mais complexo pois o output deve ter o mesmo tamanho de produto-loja.
2) Treinar vários modelos separados (considerando uma série temporal para cada modelo). Esse método é ineficiente pois o modelo nunca irá aprender os padrões entre as séries.
3) Treinar o modelo com um par produto-loja por vez. Ou seja:
   - O modelo realiza epochs = N iterações de treino ao longo de todo dataset
     - Em cada epoch, passa por todas as M batches
       - Em cada batch (que possui uma janela de tamanho input_size), atualiza os parâmetros para cada série temporal ($x_l,y_l$). Totalizando L atualizações, com L sendo o número de pares produto-loja.

Ressalta-se que cada conjunto ($x_l,y_l$) representa:
- $x_l$: série temporal do l-ésimo par produto-loja, sendo um vetor de tamanho input_size x (features + 1)
- $y_l$: Previsão de vendas do l-ésimo par produto-loja para os próximos $output_size$ dias


In [ ]:
""" Setting up the constants """

DATA_PATH = "../Data/hackathon_2025_templates/part-00000-tid-"

FILE_HASH = [
    "2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36",
    "5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8",
    "7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1",
]

SUFIX = "-4-1-c000.snappy.parquet"

FILE_NAMES = [DATA_PATH + hash + SUFIX for hash in FILE_HASH]

In [ ]:
""" Loading the data """

pdv_data = read_parquet_file(FILE_NAMES[0])
transaction_data = read_parquet_file(FILE_NAMES[1])
product_data = read_parquet_file(FILE_NAMES[2])

In [ ]:
pdv_data.head()

In [ ]:
transaction_data.head()

In [ ]:
product_data.head()

# Reorganizando os dados em pares produto-loja

In [ ]:

# 1) juntar produto
df = transaction_data.merge(
    product_data,
    left_on="internal_product_id",
    right_on="produto",
    how="left"
)

# 2) juntar loja
df = df.merge(
    pdv_data,
    left_on="internal_store_id",
    right_on="pdv",
    how="left"
)

# 3) garantir que a data esteja no formato datetime
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

# 4) pivotar: dia x par produto-loja
full_df = (
    df.pivot_table(
        index="transaction_date",                        # agora usa transaction_date
        columns=["internal_product_id", "internal_store_id"],  
        values="quantity",                               # mantenha 'quantity' se esse for o nome certo
        aggfunc="sum",
        fill_value=0
    )
)

# 5) deixar colunas mais legíveis: produto_loja
full_df.columns = [f"{p}_{l}" for p, l in full_df.columns]

# 6) garantir todas as datas do período (mesmo sem vendas)
full_df = full_df.reindex(
    pd.date_range(df["transaction_date"].min(), df["transaction_date"].max(), freq="D"),
    fill_value=0
)
full_df.index.name = "data"


# Transformando o Dataframe em um Dataset



In [ ]:
# Criação de um dataset sintético e bem comportado
# (apenas para testar o modelo, já que o real está muito bagunçado)

n_data = 1000 # Número total de pontos de dados

# Sinal composto por várias frequências + ruído
time = np.linspace(0, 20 * np.pi, n_data) # Tempo de 0 a 20π

series_1 = np.sin(time) + 0.5 * np.sin(3 * time) + 0.2 * np.sin(5 * time) + 0.1 * np.random.randn(n_data)

series_2 = np.cos(time) + 0.3 * np.sin(2 * time) + 0.4 * np.sin(4 * time) + 0.15 * np.random.randn(n_data)

# Cálculo da feature como a diferença entre o valor atual e o anterior (característica cíclica!!!)
feature_1 = np.array([0] +[series_1[t] - series_1[t-1] for t in range(1, len(series_1))])
feature_2 = np.array([0] + [series_2[t] - series_2[t-1] for t in range(1, len(series_2))])

# Normalização dos dados para a faixa [0, 1] - MÉTODO SIMPLES, PESQUISAR OUTROS
series_1 = (series_1 - np.min(series_1)) / (np.max(series_1) - np.min(series_1))
feature_1 = (feature_1 - np.min(feature_1)) / (np.max(feature_1) - np.min(feature_1))

series_2 = (series_2 - np.min(series_2)) / (np.max(series_2) - np.min(series_2))
feature_2 = (feature_2 - np.min(feature_2)) / (np.max(feature_2) - np.min(feature_2))


In [ ]:
# Inicializando os vetores de entrada e os rótulos
X = []
y = []

# Número total de amostras (janelas diferentes) que podem ser extraídas
n_samples = len(series_1) - input_size - output_size + 1

# Extraindo janelas deslizantes
for i in range(n_samples):

    X_window1 = series_1[i:i+input_size]             # janela de entrada
    X_window2 = series_2[i:i+input_size]             # janela de entrada
    feature_1window = feature_1[i:i+input_size]      # janela de entrada
    feature_2window = feature_2[i:i+input_size]      # janela de entrada

    X_window = np.stack([X_window1, X_window2, feature_1window, feature_2window], axis=1)  # shape = (input_size, 4)

    y_window1 = series_1[i+input_size:i+input_size+output_size]  # próximo dia da série 1
    y_window2 = series_2[i+input_size:i+input_size+output_size]  # próximo dia da série 2

    # Agora o rótulo também tem 2 valores (para cada série)
    y_window = np.stack([y_window1, y_window2], axis=1)  # shape = (output_size, 2)
    X.append(X_window)
    y.append(y_window)

# Convertendo para arrays numpy e depois para tensores PyTorch
X = np.array(X)  # shape = [n_samples, input_size]
y = np.array(y)  # shape = [n_samples, output_size]

print(X.shape)
X = torch.tensor(X, dtype=torch.float32)  # shape = [n_samples, input_size, 1]

y = torch.tensor(y, dtype=torch.float32)  # shape = [n_samples, 1]

" Definindo o Dataset e DataLoader do PyTorch "
class SingleSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset_full = SingleSeriesDataset(X, y)


# Separando os dados entre treino / validação

Os dados serão separados na proporção 80% - treino / 20% validação. Para séries temporais, é usual que essa separação seja feita de forma cronológica

In [ ]:
# Ponto de separação entre treino e validação
split_point = int(0.8 * len(dataset_full))

# Separação cronológica das janelas
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

num_features = X_train.shape[2]  # número de features (2 no caso: valor e tempo)
# Flatten do tensor para entrar na rede
X_train = X_train.view(X_train.shape[0], -1)  # shape = [n_samples, input_size * n_features]
X_test  = X_test.view(X_test.shape[0], -1)

print(f"Há um total de {len(dataset_full)} janelas e o split_point = {split_point}, em 80%")
print(f" O shape de X_train é {X_train.shape} e o shape de X_test é {X_test.shape}")
print(f" O shape de y_train é {y_train.shape} e o shape de y_test é {y_test.shape}")

# Treinamento

In [ ]:
# Adotando o dataset de treino
dataset = SingleSeriesDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size, shuffle=False) # shuffle=False para séries tempor

# Inicialização do modelo N-BEATS (considerando X_train com múltiplas features)
model = NBeats(input_size * num_features, hidden_size, output_size, n_layers).to(device)

model, criterion, optimizer = train_model(model, learning_rate, epochs, device, dataloader)




# Validação do modelo

São feitos dois testes:

- **Soft test:** Modelo  tenta fazer as previsões, mas não utiliza-as nas previsões futuras, utiliza sempre os *ground truth* como input
- **Hard test:** Modelo tenta fazer as previsões, e utiliza $y_{i-1}$ para a previsão de $y_i$

In [ ]:
all_preds_S, all_targets_S, avg_loss_test_S = soft_test(model, dataloader, device, criterion)

# Hard test está dando erro, conferir depois
all_preds_H, all_targets_H, avg_loss_test_H = hard_test(model, dataloader, device, criterion)

Apesar da diferença grande entre os erros do **Hard Test** e **Soft Test**, ressalta-se que, ao adicionar features exógenas aos dados não previstos (dia da semana, do mês, mês) essas features auxiliam na estabilidade do input, mesmo que o próprio modelo continue fornecendo informações imperfeitas

# Ponto de parada - RAB 18-09-25

Nosso modelo está acertando bem nos dados de treino e de teste fácil, mas (não se sabe se) ele performa mal no treino difícil, pois o erro se acumula rápido. Isso é um indicativo de:
- Se quisermos mandar bem em dados desconhecidos completamente, precisaremos de muitas features auxiliares
- O dataset de exemplo é bem comportado. Queremos então séries bem comportadas (mesmo que sejam ruidosas, mas presentes) para nossa rede. Valores de outliers podem ser:
  - Descartados
  - Previstos probabilisticamente
  - De forma trivial, falar que, para todos os outliers de jan-22, se repetirá em jan-23

No entanto, para isso precisamos entender melhor os dados, o que eu sugiro:
1) Pegar a tabelona de produto-loja e tentar filtrar aquelas que são relevantes
2) Pegar, para cada loja, os k-ésimos (1, 3, 5) produtos mais vendidos ao longo do ano, ou mais frequentes, e visualizar esses sinais
3) Ver, no geral, quais produtos são mais populares dentre todo dataset

Após fazermos essas análises, teremos uma boa ideia de que parte do nosso dataset será boa para inputar a caixa preta. Após essa escolha, pensamos nas features:
- Temporais (mais fáceis de pensar e implementar)
- Categóricas (intrínsecas do produto ou da loja, como a categoria deles)
- De localização (apenas se der tempo)

Após termos feito isso, podemos então realizar normalização dos sinais, one hot encoding do que for viável e embedding de IDs etc

In [ ]:
len(feature_1)